In [1]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import word2vec

import numpy as np
import wget
import zipfile
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD
from itertools import *
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
str(morph.parse('стали')[0].methods_stack[0][0]) == 'DictionaryAnalyzer()'

True

In [3]:
model = KeyedVectors.load_word2vec_format('araneum_upos_skipgram_300_2_2018.vec.gz')

In [4]:
all_nouns = []
for w in tqdm(model.index_to_key):
    #print(w)
    wordform, pref = w.split('_')
    if pref == 'NOUN' and len(word) > 3:
        first_res_methods = morph.parse(wordform)[0].methods_stack
        if str(first_res_methods[0][0]) == 'DictionaryAnalyzer()' and len(first_res_methods) == 1:
            all_nouns.append(w)

100%|████████████████████████████████████████████████████████████████████████| 196620/196620 [00:22<00:00, 8633.35it/s]


In [5]:
with open('araneum_all_nouns.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(all_nouns) + '\n')

In [6]:
len(all_nouns)

46074

In [66]:
with open('cosine_sim.csv', 'w', encoding='utf8') as f:
    f.write('слово1,слово2,косинус\n')

In [62]:
import math

print(math.comb(86861, 2))
print(math.comb(67538, 2))
print(math.comb(46074, 2))

3772373230
2280656953
1061383701


In [20]:
cosine_similarity([model['год_NOUN']], [model['человек_NOUN']])[0][0]

0.20726758

In [ ]:
cosine_sim = []

num_comb = 0
for word1, word2 in tqdm(combinations(all_nouns, 2), total=2280656953):
    cos_words = cosine_similarity([model[word1]], [model[word2]])[0][0]
    if cos_words >= 0.6:
        cosine_sim.append(f'{word1},{word2},{str(cos_words)}')
        num_comb += 1
        
        if num_comb % 10 == 0:
            with open('cosine_sim.csv', 'a', encoding='utf8') as f:
                f.write('\n'.join(cosine_sim) + '\n')
            cosine_sim = []

  6%|███▋                                                      | 145045549/2280656953 [20:17:56<113:18:34, 5235.44it/s]

In [18]:
cosine_sim

[['год_NOUN', 'год_NOUN', array([[0.9999998]], dtype=float32)],
 ['год_NOUN', 'человек_NOUN', array([[1.]], dtype=float32)],
 ['год_NOUN', 'время_NOUN', array([[0.99999994]], dtype=float32)],
 ['год_NOUN', 'дело_NOUN', array([[1.0000002]], dtype=float32)],
 ['год_NOUN', 'день_NOUN', array([[0.9999999]], dtype=float32)],
 ['год_NOUN', 'жизнь_NOUN', array([[1.]], dtype=float32)],
 ['год_NOUN', 'рука_NOUN', array([[1.0000001]], dtype=float32)],
 ['год_NOUN', 'слово_NOUN', array([[0.99999994]], dtype=float32)],
 ['год_NOUN', 'раз_NOUN', array([[1.0000001]], dtype=float32)],
 ['год_NOUN', 'глаз_NOUN', array([[1.]], dtype=float32)],
 ['год_NOUN', 'лицо_NOUN', array([[1.]], dtype=float32)],
 ['год_NOUN', 'место_NOUN', array([[0.99999994]], dtype=float32)],
 ['год_NOUN', 'работа_NOUN', array([[1.0000001]], dtype=float32)],
 ['год_NOUN', 'голова_NOUN', array([[0.99999994]], dtype=float32)],
 ['год_NOUN', 'сторона_NOUN', array([[0.99999994]], dtype=float32)],
 ['год_NOUN', 'друг_NOUN', array([[1

In [23]:
model.most_similar('год_NOUN')

[('год_PROPN', 0.7849781513214111),
 ('месяц_NOUN', 0.6693861484527588),
 ('гектодар_NOUN', 0.6445382833480835),
 ('десятилетие_NOUN', 0.6254161596298218),
 ('годемик_NOUN', 0.5993143320083618),
 ('столетие_NOUN', 0.566545844078064),
 ('го-д_NOUN', 0.562800407409668),
 ('г.од_NOUN', 0.5580312013626099),
 ('неделя_NOUN', 0.5551120638847351),
 ('лт_PROPN', 0.5386587381362915)]